### Module Import

In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D
from tensorflow.python.client import device_lib
import keras.backend.tensorflow_backend as K
from keras import layers, models

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

### Raw data reading

In [2]:
raw_data = pd.read_csv('Rawdata.csv').drop(['Unnamed: 0', 'Unnamed: 0.1'], 
                                           axis=1)
raw_data

,Author,HotelInfo.HotelID,Service,Cleanliness,Value,Sleep Quality,Rooms,Location,Content,Date
0,Clara79,280518,3,3,4,4.0,3.0,3.0,"['hotel', 'bit', 'surprise', 'could', 'find', ...","September 22, 2004"
1,BerlinerPooh,280518,1,1,1,1.0,0.0,1.0,"['room', 'adequate', 'little', 'drab', 'side',...","January 18, 2005"
2,amazingthing,280518,2,4,3,4.0,3.0,3.0,"['first', 'read', 'review', 'side', 'booking',...","May 10, 2005"
3,Nitus,280518,4,4,5,5.0,0.0,0.0,"['one', 'night', 'stay', 'artus', 'hotel', 'be...","October 23, 2005"
4,mattp1874,280518,4,5,4,5.0,4.0,5.0,"['stayed', 'hotel', 'night', 'ok', 'missing', ...","March 12, 2007"
...,...,...,...,...,...,...,...,...,...,...
355167,Posted by an Accorhotels.com traveler,289861,3,4,3,0.0,4.0,2.0,"['stayed', 'week', 'rustica', 'child', 'year',...","October 28, 2011"
355168,Posted by an Accorhotels.com traveler,289861,3,4,5,0.0,4.0,1.0,"['superb', 'stay', 'novotel', 'although', 'out...","November 4, 2011"
355169,paulpowell3540,289861,4,5,4,0.0,5.0,0.0,"['yes', 'distance', 'rome', 'however', 'hotel'...","November 10, 2011"
355170,uclabobby,289861,5,5,5,0.0,5.0,0.0,"['hotel', 'nice', 'look', 'nice', 'room', 'com...","January 2, 2012"


In [3]:
content_list = np.array(raw_data['Content'].tolist())
content_list

array(["['hotel', 'bit', 'surprise', 'could', 'find', 'review', 'anywhere', 'however', 'turned', 'entirely', 'pleasant', 'one', 'location', 'great', 'close', 'ku', 'damn', 'charlottenburg', 'literally', 'next', 'berliner', 'strasse', 'u', 'bahn', 'staff', 'excellent', 'willing', 'help', 'bunch', 'english', 'direction', 'good', 'restaurant', 'club', 'hour', 'room', 'huge', 'expected', 'bedroom', 'bathroom', 'got', 'luxurious', 'lounge', 'little', 'kitchen', 'would', 'highly', 'recommend', 'hotel', 'anyone', 'paid', 'le', 'twin', 'room', 'pity', 'could', 'stay', 'longer', 'appreciate', 'berlin', 'great', 'hotel']",
       "['room', 'adequate', 'little', 'drab', 'side', 'noisy', 'main', 'street', 'front', 'kitchenette', 'removed', 'used', 'area', 'storing', 'luggage', 'shopping', 'underground', 'road', 'another', 'across', 'street', 'take', 'time', 'reach', 'fabulous', 'shop', 'restaurant', 'busy', 'area', 'ku', 'damm', 'main', 'entrance', 'hotel', 'rather', 'grand', 'adjacent', 'large', 

### Data Tokenizing & word_indexing

In [4]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(content_list)
tokenizer.word_index

{"'room'": 1,
 "'hotel'": 2,
 "'great'": 3,
 "'stay'": 4,
 "'would'": 5,
 "'staff'": 6,
 "'good'": 7,
 "'night'": 8,
 "'one'": 9,
 "'time'": 10,
 "'u'": 11,
 "'nice'": 12,
 "'day'": 13,
 "'stayed'": 14,
 "'get'": 15,
 "'location'": 16,
 "'clean'": 17,
 "'service'": 18,
 "'breakfast'": 19,
 "'bed'": 20,
 "'pool'": 21,
 "'also'": 22,
 "'place'": 23,
 "'restaurant'": 24,
 "'like'": 25,
 "'area'": 26,
 "'well'": 27,
 "'really'": 28,
 "'could'": 29,
 "'back'": 30,
 "'go'": 31,
 "'small'": 32,
 "'bathroom'": 33,
 "'friendly'": 34,
 "'food'": 35,
 "'walk'": 36,
 "'view'": 37,
 "'floor'": 38,
 "'even'": 39,
 "'two'": 40,
 "'check'": 41,
 "'desk'": 42,
 "'minute'": 43,
 "'got'": 44,
 "'helpful'": 45,
 "'resort'": 46,
 "'comfortable'": 47,
 "'front'": 48,
 "'much'": 49,
 "'little'": 50,
 "'bar'": 51,
 "'first'": 52,
 "'free'": 53,
 "'people'": 54,
 "'price'": 55,
 "'next'": 56,
 "'around'": 57,
 "'lot'": 58,
 "'right'": 59,
 "'trip'": 60,
 "'beach'": 61,
 "'thing'": 62,
 "'best'": 63,
 "'excelle

In [5]:
ipc_Xdata = np.array(tokenizer.texts_to_sequences(content_list))
print(ipc_Xdata)
print(ipc_Xdata.shape)

[list([2, 85, 768, 29, 107, 89, 478, 100, 583, 2917, 292, 9, 16, 3, 69, 8527, 5183, 16683, 829, 56, 15841, 9789, 11, 3313, 6, 64, 1109, 279, 2036, 466, 610, 7, 24, 270, 97, 1, 199, 399, 235, 33, 44, 980, 361, 50, 368, 5, 309, 65, 2, 312, 252, 254, 973, 1, 4054, 29, 4, 840, 1629, 1730, 3, 2])
 list([1, 569, 50, 4249, 157, 404, 208, 66, 48, 1354, 1807, 194, 26, 4084, 355, 234, 1264, 350, 124, 211, 66, 87, 10, 1390, 526, 212, 24, 342, 26, 8527, 9059, 208, 607, 2, 406, 379, 1366, 84, 4581, 51, 26, 19, 196, 267, 6, 34, 587, 50, 667, 24, 43, 70, 552, 63, 867, 178, 1201, 101, 68, 30])
 list([52, 341, 89, 157, 392, 138, 114, 298, 1, 44, 9, 207, 70, 208, 350, 140, 300, 50, 99, 235, 562, 585, 1, 44, 1354, 27, 243, 1, 8, 746, 2, 1730, 103, 375, 1877, 2, 85, 2063, 533, 56, 273, 136, 87, 256, 1633, 16256, 17918, 34760, 1514, 2, 6, 45, 5, 2698, 296, 56, 1730, 4, 2])
 ...
 list([372, 197, 445, 100, 2, 420, 375, 289, 181, 3190, 259, 10, 13, 479, 445, 1, 1220, 17, 3, 33, 1904, 26, 556, 2823, 796, 100, 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [6]:
ipc_Ydata = np.array(raw_data[['Service', 'Cleanliness','Value',
                               'Sleep Quality','Rooms','Location']])
print(ipc_Ydata)
print(ipc_Ydata.shape)

[[3. 3. 4. 4. 3. 3.]
 [1. 1. 1. 1. 0. 1.]
 [2. 4. 3. 4. 3. 3.]
 ...
 [4. 5. 4. 0. 5. 0.]
 [5. 5. 5. 0. 5. 0.]
 [4. 5. 3. 0. 5. 0.]]
(355172, 6)


### Data Split & Get maxlen and max_features

In [7]:
train_x, test_x, train_y, test_y = train_test_split(ipc_Xdata, ipc_Ydata, 
                                                    test_size=0.3)
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

(248620,) (106552,) (248620, 6) (106552, 6)


In [8]:
maxlen = 1
max_features = 1
for idx in ipc_Xdata :
    try :
        len_ = len(idx) #print(idx)
        max_ = np.max(idx)
    except :
        #print(idx)
        continue
    #print(type(max_))
    #break
    if max_features < max_ :
        max_features = max_
    if maxlen < len_ :
        maxlen = len_
print(maxlen)
print(max_features)

2670
148649


### Model building

#### variables setting

In [9]:
# Embedding
max_features = 148650 # feature 값 변경
maxlen = 2670 # 리뷰의 길이
embedding_size = 64

# Convolution
kernel_size = 2
filters = 256
pool_size = 4

# LSTM
lstm_output_size = 10

# Training
batch_size = 200 #30 -> 20
epochs = 100

#### Padding

In [10]:
print('Loading data...')

print(len(train_x), 'train sequences')
print(len(test_x), 'test sequences')

print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

print('Pad sequences (samples x time)')

train_x = sequence.pad_sequences(train_x, maxlen=maxlen)
test_x = sequence.pad_sequences(test_x, maxlen=maxlen)
print('train_x shape:', train_x.shape)
print('test_x shape:', test_x.shape)

Loading data...
248620 train sequences
106552 test sequences
(248620,) (106552,) (248620, 6) (106552, 6)
Pad sequences (samples x time)
train_x shape: (248620, 2670)
test_x shape: (106552, 2670)


#### Model Building

In [11]:
print('Build model...')
#model = Sequential()
input_layer = layers.Input((maxlen,))
embedding_layer = layers.Embedding(max_features, embedding_size)(input_layer)

convolution_layer = layers.Conv1D(64, 3, padding = 'valid',
                                 activation='relu', strides=5)(embedding_layer)
maxpooling_layer = layers.MaxPooling1D(pool_size)(convolution_layer)

lstm_layer = layers.Bidirectional(layers.CuDNNLSTM(64, return_sequences=True))(maxpooling_layer)

dropout_layer = layers.Dropout(0.3)(lstm_layer) 

flatten_layer = layers.Flatten()(dropout_layer)
#model.add(Embedding(max_features, embedding_size, input_length=maxlen))

#model.add(MaxPooling1D(pool_size=pool_size))

#model.add(Dropout(0.25))

#model.add(Conv1D(embedding_size,
#             kernel_size,
#             padding='valid',
#             activation='relu',
#             strides=2))

#model.add(LSTM(lstm_output_size))
#model.add(MaxPooling1D(pool_size=pool_size))
output_layer = layers.Dense(6, activation = 'sigmoid')(flatten_layer)
#model.add(Activation('sigmoid'))
model = models.Model(inputs=input_layer, outputs=output_layer)
model.summary()
#loss binary_crossentropy
model.compile(loss='mse',
              optimizer='adam',
              metrics=['accuracy'])

Build model...





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2670)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 2670, 64)          9513600   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 534, 64)           12352     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 133, 64)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 133, 128)          66560     
_________________________________________________________________
dropout_1 (Dropout)          (None, 133, 128)          0

#### Train

In [12]:
with K.tf.device('/gpu:0'):
    print('Train...')
    model.fit(train_x, train_y / 5,
              batch_size=batch_size,
              epochs=epochs)
    score, acc = model.evaluate(test_x, test_y, batch_size=batch_size)

    print('Test score:', score)
    print('Test accuracy:', acc)

Train...

Epoch 1/100
248620/248620 [==============================] - 36s 146us/step - loss: 0.1332 - acc: 0.1858
Epoch 2/100
248620/248620 [==============================] - 34s 139us/step - loss: 0.1281 - acc: 0.1866
Epoch 3/100
248620/248620 [==============================] - 35s 139us/step - loss: 0.1155 - acc: 0.1872
Epoch 4/100
248620/248620 [==============================] - 35s 139us/step - loss: 0.0966 - acc: 0.1866
Epoch 5/100
131400/248620 [==============>...............] - ETA: 16s - loss: 0.0794 - acc: 0.1844

KeyboardInterrupt: 